In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("ggplot")

import shapely

In [2]:
blk_shape = gpd.read_file("shapefiles/tabblock2010_37_pophu.shp")
cd112 = pd.read_csv("BlockAssign_ST37_NC_CD.txt")
cd113 = pd.read_csv("shapefiles/37_NC_CD113.txt")
cd115 = pd.read_csv("shapefiles/37_NC_CD115.txt")

In [3]:
cd112["CD112"] = cd112["DISTRICT"]
cd112.drop(columns="DISTRICT", inplace=True)

In [4]:
blk_shape["BLOCKID"] = pd.to_numeric(blk_shape["BLOCKID10"])

In [5]:
df = blk_shape.copy()
df.crs = blk_shape.crs
df = pd.merge(df, cd113, on="BLOCKID", how="left")
df.drop(columns=["BLOCKID10", "PARTFLG", "HOUSING10"], inplace=True)
df = pd.merge(df, cd115, on="BLOCKID", how="left")
df = pd.merge(df, cd112, on="BLOCKID", how="left")

In [6]:
nc_proj = "+proj=lcc +lat_1=34.33333333333334 +lat_2=36.16666666666666 +lat_0=33.75 +lon_0=-79 +x_0=609601.2199999999 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs "
df.to_crs(nc_proj, inplace=True)

In [7]:
df["centroid"] = df["geometry"].centroid

In [8]:
df['x'] = [p.x for p in df["centroid"]]
df['y'] = [p.y for p in df["centroid"]]

norm = max(df['x']) - min(df['x'])
df['x'] /= norm
df['y'] /= norm
df['x'] -= min(df['x'])
df['y'] -= min(df['y'])

# normalized to 0-1 on x-axis, preserving scale between x and y

In [13]:
df = df[["STATEFP10", "COUNTYFP10", "TRACTCE10", "BLOCKCE", "geometry", "POP10", "CD112", "CD113", "CD115"]]

In [ ]:
df.to_file("data_full.shp")